# Animal Welfare and Stress Detection System

**Course:** Pattern Recognition  
**Project Type:** Final Group Project (Part 2)  
**Dataset:** Animals-10 (Kaggle)

---


### Team Members & Roles

- **Technical Lead:**  
  *Name:* Ashwin Shrestha  
  *Responsibilities:* System design, model implementation, experimental evaluation, Definition and validation of five (5) publication-ready research questions, final submission on Teams.

- **Figures, Tables & Presentation**  
  *Name:* Arpan Rai 
  *Responsibilities:* Designs all figures,Prepares and maintains the presentation slides

- **Student 3 – Report & Storytelling:**  
  *Name:* Ashwin Shrestha/Arpan Rai 
  *Responsibilities:*Writing the final report using the provided Overleaf template, ensuring narrative coherence across sections, aligning methodology, results, and discussion.

---

## Project Overview

This project presents a **vision-based animal welfare and stress-risk screening system** built using the Animals-10 dataset.  
The system does **not diagnose physiological stress directly**; instead, it infers **visual indicators associated with potential stress or welfare compromise**, such as posture and body compactness, and combines them with species recognition and explainability techniques to support human decision-making.

The project addresses five research questions (RQ1–RQ5) through structured experiments, quantitative evaluation, and interpretable outputs.


## Project Summary and Research Questions

### Project Summary

Animal welfare assessment is a critical task in domains such as farming, veterinary care, and companion animal monitoring. In many real-world settings, welfare and stress cannot be measured directly and must instead be inferred from **visual behavioral indicators**, including posture, body compactness, and activity-related cues.

This project proposes a **vision-based animal welfare and stress-risk screening system** using the Animals-10 image dataset. The system leverages convolutional neural networks for species recognition, simple posture-based visual proxies for potential stress indicators, multimodal feature fusion, robustness testing under uncertainty, and explainable decision-support mechanisms. The goal is to demonstrate how pattern recognition techniques can support **early-stage welfare screening**, rather than clinical diagnosis.

---

### Research Questions

**RQ1:**  
*How accurately can a convolutional neural network classify animal species from images in the Animals-10 dataset?*

**RQ2:**  
*Can posture-related visual features be extracted from images and summarized in a way that is meaningful for animal welfare analysis?*

**RQ3:**  
*Does fusing posture-based features with species-level appearance probabilities improve stress-risk proxy inference compared to unimodal approaches?*

**RQ4:**  
*How robust is the proposed system under missing or noisy visual information that resembles real-world monitoring conditions?*

**RQ5:**  
*Do explainable AI techniques and simple rule-based reasoning improve the interpretability and practical usefulness of welfare and stress-risk predictions for end users?*

---

> **Note:**  
> The Animals-10 dataset does not contain ground-truth stress labels. Therefore, this work focuses on **stress-risk proxies derived from visual posture and appearance cues**, and the results are intended for **screening and decision-support purposes**, not physiological stress diagnosis.


In [1]:
# =========================
# Cell C: Setup & Imports
# =========================

import os, zipfile, subprocess, random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
# Output structure
BASE_DIR = Path("outputs_part2_animals10")
FIG_ROOT = BASE_DIR / "Figures_Tables"

for rq in ["RQ1","RQ2","RQ3","RQ4","RQ5"]:
    (FIG_ROOT / rq).mkdir(parents=True, exist_ok=True)

print("Working directory:", Path(".").resolve())
print("Output folder:", FIG_ROOT.resolve())

Working directory: C:\Users\nitro\Documents\animals10_project
Output folder: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables


In [2]:
# =========================
# Cell D: Dataset Download
# =========================

# Dataset info
KAGGLE_DATASET = "alessiocorrado99/animals10"
DATA_DIR = Path("data_animals10")

# Check kaggle.json
kaggle_path = Path.home() / ".kaggle" / "kaggle.json"
if not kaggle_path.exists():
    raise FileNotFoundError(
        "kaggle.json not found. "
        "Place kaggle.json in ~/.kaggle/ and restart the kernel."
    )

# Create data directory
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Download dataset if not already present
zip_files = list(DATA_DIR.glob("*.zip"))
if not zip_files:
    print("Downloading Animals-10 dataset from Kaggle...")
    subprocess.run(
        ["kaggle", "datasets", "download", "-d", KAGGLE_DATASET, "-p", str(DATA_DIR)],
        check=True
    )
else:
    print("Dataset zip already exists. Skipping download.")

# Extract dataset
for zip_path in DATA_DIR.glob("*.zip"):
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(DATA_DIR)
    print(f"Extracted {zip_path.name}")

print("✅ Dataset ready at:", DATA_DIR.resolve())


Extracted animals10.zip
✅ Dataset ready at: C:\Users\nitro\Documents\animals10_project\data_animals10


In [3]:
# =========================
# Cell E: Locate Image Root
# =========================

from torchvision.datasets import ImageFolder

def find_image_root(base_dir: Path):
   
    for p in base_dir.rglob("*"):
        if p.is_dir():
            subdirs = [d for d in p.iterdir() if d.is_dir()]
            if len(subdirs) >= 8:  # Animals-10 has 10 classes
                # quick sanity check: folder names are non-empty strings
                if all(len(d.name) > 0 for d in subdirs):
                    return p
    return None

IMG_ROOT = find_image_root(DATA_DIR)

if IMG_ROOT is None:
    raise RuntimeError("Could not locate image root directory.")
else:
    print("✅ Image root found at:")
    print(IMG_ROOT.resolve())

tmp_ds = ImageFolder(IMG_ROOT)
print("Classes found:", tmp_ds.classes)
print("Number of classes:", len(tmp_ds.classes))


✅ Image root found at:
C:\Users\nitro\Documents\animals10_project\data_animals10\raw-img
Classes found: ['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo']
Number of classes: 10


In [4]:
# =========================
# Cell F: Transforms + Split + DataLoaders
# =========================

import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

# Force CPU for reliability (your GPU shows sm_120 incompatibility with your torch build)
device = torch.device("cpu")
print("✅ Using device:", device)

# Parameters (CPU-friendly)
IMG_SIZE = 128        # 128 is fast; change to 224 if you have time
BATCH_SIZE = 32
SUBSET_N = 3000       # increase (e.g., 6000) if your laptop is fast

# Transforms
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# Base dataset for length/classes
base_ds = datasets.ImageFolder(IMG_ROOT, transform=eval_tfms)
class_names = base_ds.classes
n_classes = len(class_names)
N = len(base_ds)

print("Classes:", class_names)
print("Total images:", N)

# Subset indices (CPU speed-up)
rng = np.random.default_rng(SEED)
subset_idx = rng.choice(N, size=min(SUBSET_N, N), replace=False).tolist()
random.shuffle(subset_idx)

# Split indices
n_total = len(subset_idx)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val

train_idx = subset_idx[:n_train]
val_idx   = subset_idx[n_train:n_train+n_val]
test_idx  = subset_idx[n_train+n_val:]

print("Split sizes:", n_train, n_val, n_test)

# IMPORTANT: separate datasets so transforms are correct
train_full = datasets.ImageFolder(IMG_ROOT, transform=train_tfms)
eval_full  = datasets.ImageFolder(IMG_ROOT, transform=eval_tfms)

train_ds = Subset(train_full, train_idx)
val_ds   = Subset(eval_full, val_idx)
test_ds  = Subset(eval_full, test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dl  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print("✅ DataLoaders ready:")
print("train:", len(train_ds), "val:", len(val_ds), "test:", len(test_ds))


✅ Using device: cpu
Classes: ['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo']
Total images: 26179
Split sizes: 2100 450 450
✅ DataLoaders ready:
train: 2100 val: 450 test: 450


## RQ1: Species recognition baseline (CNN)

This section evaluates a transfer-learning CNN model (ResNet-18) for classifying the 10 animal categories in the Animals-10 dataset. Species recognition is a prerequisite for downstream welfare-oriented analysis because visual indicators and posture cues can differ across animal species.


In [5]:
# =========================
# RQ1: Train ResNet18 
# =========================

import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import accuracy_score, f1_score

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, n_classes)

# Freeze backbone for speed
for p in model.parameters():
    p.requires_grad = False
for p in model.fc.parameters():
    p.requires_grad = True

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

def run_epoch(dl, train=True):
    model.train(train)
    y_true, y_pred = [], []
    losses = []

    for xb, yb in dl:
        xb, yb = xb.to(device), yb.to(device)

        if train:
            optimizer.zero_grad()

        out = model(xb)
        loss = criterion(out, yb)

        if train:
            loss.backward()
            optimizer.step()

        losses.append(loss.item())
        preds = out.argmax(1)

        y_true.extend(yb.detach().cpu().numpy().tolist())
        y_pred.extend(preds.detach().cpu().numpy().tolist())

    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    return float(np.mean(losses)), float(acc), float(mf1)

EPOCHS = 2
history = []

for ep in range(1, EPOCHS + 1):
    tr_loss, tr_acc, tr_f1 = run_epoch(train_dl, train=True)
    va_loss, va_acc, va_f1 = run_epoch(val_dl, train=False)

    history.append([ep, tr_loss, tr_acc, tr_f1, va_loss, va_acc, va_f1])
    print(f"Epoch {ep}: train acc={tr_acc:.3f} f1={tr_f1:.3f} | val acc={va_acc:.3f} f1={va_f1:.3f}")

hist_df = pd.DataFrame(history, columns=[
    "epoch","train_loss","train_acc","train_f1","val_loss","val_acc","val_f1"
])
hist_df


Epoch 1: train acc=0.560 f1=0.488 | val acc=0.784 f1=0.750
Epoch 2: train acc=0.796 f1=0.773 | val acc=0.840 f1=0.826


,epoch,train_loss,train_acc,train_f1,val_loss,val_acc,val_f1
0,1,1.391590,0.56000,0.488037,0.744297,0.784444,0.750012
1,2,0.698279,0.79619,0.772784,0.537972,0.840000,0.826238


In [6]:
# =========================
# RQ1: Outputs
# =========================

import torch
from sklearn.metrics import confusion_matrix, classification_report

# ---- Evaluate on test set ----
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for xb, yb in test_dl:
        xb = xb.to(device)
        out = model(xb)
        preds = out.argmax(1).cpu().numpy().tolist()
        y_pred.extend(preds)
        y_true.extend(yb.numpy().tolist())

test_acc = accuracy_score(y_true, y_pred)
test_f1  = f1_score(y_true, y_pred, average="macro")
cm = confusion_matrix(y_true, y_pred)

print("RQ1 Test Accuracy:", test_acc)
print("RQ1 Test Macro-F1:", test_f1)
print("Confusion matrix total:", cm.sum(), "| diagonal correct:", cm.diagonal().sum())

# ---- Figure 1: Training curve ----
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(hist_df["epoch"], hist_df["train_acc"], marker="o", label="train_acc")
ax.plot(hist_df["epoch"], hist_df["val_acc"], marker="o", label="val_acc")
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy")
ax.set_title("RQ1: Training curve (ResNet18 pretrained, frozen backbone)")
ax.set_xticks(hist_df["epoch"].tolist())
ax.grid(True, linestyle="--", alpha=0.3)
ax.legend()
fig.savefig(FIG_ROOT/"RQ1"/"RQ1_Fig1.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Figure 2: Confusion matrix ----
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111)
ax.imshow(cm)
ax.set_title("RQ1: Confusion matrix (test set)")
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_xticks(range(n_classes))
ax.set_yticks(range(n_classes))
ax.set_xticklabels(class_names, rotation=45, ha="right")
ax.set_yticklabels(class_names)

for (i, j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha="center", va="center", fontsize=6)

fig.savefig(FIG_ROOT/"RQ1"/"RQ1_Fig2.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Table 1: Summary + training history ----
rq1_tab1 = pd.DataFrame([{
    "Model": "ResNet18 pretrained (frozen backbone)",
    "IMG_SIZE": IMG_SIZE,
    "SUBSET_N": len(train_ds) + len(val_ds) + len(test_ds),
    "Epochs": int(hist_df["epoch"].max()),
    "Test_Accuracy": float(test_acc),
    "Test_MacroF1": float(test_f1),
}])

with pd.ExcelWriter(FIG_ROOT/"RQ1"/"RQ1_Tab1.xlsx") as w:
    rq1_tab1.to_excel(w, index=False, sheet_name="Summary")
    hist_df.to_excel(w, index=False, sheet_name="TrainingHistory")

# ---- Table 2: Per-class classification report ----
rep = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
rq1_tab2 = (pd.DataFrame(rep).T
            .rename(columns={"f1-score":"f1", "support":"support_n"})
            .reset_index()
            .rename(columns={"index":"class"}))

with pd.ExcelWriter(FIG_ROOT/"RQ1"/"RQ1_Tab2.xlsx") as w:
    rq1_tab2.to_excel(w, index=False, sheet_name="ClassReport")

print("✅ Saved RQ1_Fig1.pdf, RQ1_Fig2.pdf, RQ1_Tab1.xlsx, RQ1_Tab2.xlsx in:", (FIG_ROOT/"RQ1").resolve())


RQ1 Test Accuracy: 0.8177777777777778
RQ1 Test Macro-F1: 0.7829122181754088
Confusion matrix total: 450 | diagonal correct: 368
✅ Saved RQ1_Fig1.pdf, RQ1_Fig2.pdf, RQ1_Tab1.xlsx, RQ1_Tab2.xlsx in: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables\RQ1


## RQ2: Extract posture-related visual proxy features

This section extracts simple posture-related **visual proxy features** from animal images (e.g., normalized body height, bounding-box aspect ratio, and a compactness-based crouch score). These features are not ground-truth stress labels, but they serve as interpretable indicators that can support welfare-oriented screening.


In [7]:
# =========================
# RQ2: Posture proxy feature extraction
# =========================

from PIL import Image
from torch.utils.data import Subset

# Robust helper: works for ImageFolder, Subset(ImageFolder), Subset(Subset(...)), etc.
def get_path_label(ds, i: int):
    """
    Returns (path, label) for datasets that may be wrapped in one or more Subset objects.
    """
    while isinstance(ds, Subset):
        i = ds.indices[i]
        ds = ds.dataset
    if not hasattr(ds, "samples"):
        raise TypeError(f"Expected ImageFolder-like dataset with .samples, got: {type(ds)}")
    path, label = ds.samples[i]
    return path, label

def posture_features(img_path: str):
    """
    Simple, fast posture proxy from a single image:
    - norm_h: approximate foreground height normalized by image height
    - aspect: approximate bbox width/height
    - crouch_score: heuristic combining low height + compactness
    """
    img = Image.open(img_path).convert("L")
    g = (np.array(img).astype(np.float32) / 255.0)

    # foreground mask (simple threshold)
    mask = (g < 0.85).astype(np.uint8)
    ys, xs = np.where(mask > 0)

    # fallback for near-empty masks
    if len(xs) < 80:
        return {"norm_h": 1.0, "aspect": 1.0, "crouch_score": 0.5}

    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    bw = float(x1 - x0 + 1)
    bh = float(y1 - y0 + 1)
    H, W = g.shape

    aspect = bw / (bh + 1e-6)
    norm_h = bh / (H + 1e-6)

    # crouch_score: higher when foreground height is small + bbox is compact
    crouch_score = float((1.0 - norm_h) * 0.7 + (np.clip(aspect, 0, 3) / 3.0) * 0.3)

    return {"norm_h": norm_h, "aspect": aspect, "crouch_score": crouch_score}

# Extract features from a subset of the TEST set for speed
FEAT_N = min(1200, len(test_ds))
rows = []

for i in range(FEAT_N):
    path, lab = get_path_label(test_ds, i)
    f = posture_features(path)
    rows.append({
        "path": path,
        "species": class_names[lab],
        **f
    })

feat_df = pd.DataFrame(rows)
print("✅ feat_df created:", feat_df.shape)
feat_df.head()


✅ feat_df created: (450, 5)


,path,species,norm_h,aspect,crouch_score
0,data_animals10\raw-img\pecora\OIP-AqQfFyxS3j2x...,pecora,1.0,1.071429,0.107143
1,data_animals10\raw-img\mucca\OIP-jYcEj5NnKXwRy...,mucca,1.0,1.829268,0.182927
2,data_animals10\raw-img\pecora\OIP-L47z6AcTVbth...,pecora,1.0,1.333333,0.133333
3,data_animals10\raw-img\ragno\OIP-N9pNu0AMQ6deV...,ragno,1.0,1.167315,0.116732
4,data_animals10\raw-img\cavallo\OIP-gR_Md9sVfB7...,cavallo,1.0,0.803333,0.080333


In [8]:
# =========================
# RQ2: Outputs (2 figures + 2 tables)
# =========================

# ---- Figure 1: Mean crouch score by species ----
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)

mean_crouch = feat_df.groupby("species")["crouch_score"].mean().sort_values()
ax.barh(mean_crouch.index, mean_crouch.values)
ax.set_xlabel("Mean crouch score (proxy)")
ax.set_title("RQ2: Mean posture-based crouch score by species")

fig.savefig(FIG_ROOT/"RQ2"/"RQ2_Fig1.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Figure 2: Aspect ratio vs normalized height scatter ----
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)

for sp in sorted(feat_df["species"].unique()):
    tmp = feat_df[feat_df["species"] == sp]
    ax.scatter(tmp["aspect"], tmp["norm_h"], s=10, alpha=0.35, label=sp)

ax.set_xlabel("Aspect ratio (bbox width / height)")
ax.set_ylabel("Normalized height (bbox height / image height)")
ax.set_title("RQ2: Posture proxy feature space (subset)")
ax.legend(fontsize=7, ncols=2)

fig.savefig(FIG_ROOT/"RQ2"/"RQ2_Fig2.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Table 1: Mean posture features by species ----
rq2_tab1 = (feat_df
            .groupby("species")[["norm_h","aspect","crouch_score"]]
            .mean()
            .reset_index())

with pd.ExcelWriter(FIG_ROOT/"RQ2"/"RQ2_Tab1.xlsx") as w:
    rq2_tab1.to_excel(w, index=False, sheet_name="Mean_Features")

# ---- Table 2: Descriptive statistics (mean/std/median) ----
rq2_tab2 = (feat_df
            .groupby("species")[["norm_h","aspect","crouch_score"]]
            .agg(["mean","std","median"])
            .reset_index())

# Flatten column names
rq2_tab2.columns = ["species"] + [f"{a}_{b}" for a,b in rq2_tab2.columns[1:]]

with pd.ExcelWriter(FIG_ROOT/"RQ2"/"RQ2_Tab2.xlsx") as w:
    rq2_tab2.to_excel(w, index=False, sheet_name="Descriptive_Stats")

print("✅ Saved RQ2_Fig1.pdf, RQ2_Fig2.pdf, RQ2_Tab1.xlsx, RQ2_Tab2.xlsx in:",
      (FIG_ROOT/"RQ2").resolve())


✅ Saved RQ2_Fig1.pdf, RQ2_Fig2.pdf, RQ2_Tab1.xlsx, RQ2_Tab2.xlsx in: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables\RQ2


## RQ3: Does fusing posture and appearance improve stress-risk proxy inference?

The Animals-10 dataset does not provide ground-truth stress labels. Therefore, this section constructs a **stress-risk proxy** using posture-based indicators (crouch score) within each species. We compare three models:

1) **Posture-only** (visual proxy features)  
2) **Appearance-only** (CNN class probability vector)  
3) **Fused** (posture + CNN probabilities)

We evaluate each model using Accuracy, F1 score, and ROC-AUC on the same train/test split.


In [9]:
# =========================
# RQ3: Meta-feature dataset + unimodal vs fused models
# =========================

import torch
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Inference transform (self-contained; avoids NameError)
infer_tfm = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

def cnn_probs(img_path: str):
    img = Image.open(img_path).convert("RGB")
    x = infer_tfm(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        prob = torch.softmax(out, dim=1).cpu().numpy().reshape(-1)
    return prob

# Build meta dataset from feat_df (posture) + CNN probabilities (appearance)
META_N = min(600, len(feat_df))  # keep small for speed
meta_rows = []

for i in range(META_N):
    r = feat_df.iloc[i]
    prob = cnn_probs(r["path"])
    meta_rows.append({
        "species": r["species"],
        "crouch_score": float(r["crouch_score"]),
        "norm_h": float(r["norm_h"]),
        "aspect": float(r["aspect"]),
        **{f"p_{class_names[j]}": float(prob[j]) for j in range(n_classes)}
    })

meta_df = pd.DataFrame(meta_rows)
print("✅ meta_df created:", meta_df.shape)

# Stress-risk proxy: within each species, top 30% crouch_score -> "stress_proxy" = 1
meta_df["stress_proxy"] = 0
for sp in meta_df["species"].unique():
    q = meta_df.loc[meta_df["species"] == sp, "crouch_score"].quantile(0.70)
    meta_df.loc[(meta_df["species"] == sp) & (meta_df["crouch_score"] >= q), "stress_proxy"] = 1

# Feature matrices
X_posture = meta_df[["crouch_score","norm_h","aspect"]].to_numpy()
X_species = meta_df[[c for c in meta_df.columns if c.startswith("p_")]].to_numpy()
X_fused   = np.concatenate([X_posture, X_species], axis=1)
y = meta_df["stress_proxy"].to_numpy()

# Split indices (stratified)
tr_idx, te_idx = train_test_split(
    np.arange(len(y)), test_size=0.25, random_state=SEED, stratify=y
)

def train_eval(X, name):
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(max_iter=400))
    ])
    clf.fit(X[tr_idx], y[tr_idx])
    pred = clf.predict(X[te_idx])
    proba = clf.predict_proba(X[te_idx])[:, 1]
    return clf, pred, proba, {
        "Model": name,
        "Accuracy": float(accuracy_score(y[te_idx], pred)),
        "F1": float(f1_score(y[te_idx], pred)),
        "ROC-AUC": float(roc_auc_score(y[te_idx], proba)),
    }

clf_post, pred_post, proba_post, row_post = train_eval(X_posture, "Posture-only")
clf_spec, pred_spec, proba_spec, row_spec = train_eval(X_species, "Appearance-only (CNN probs)")
clf_fuse, pred_fuse, proba_fuse, row_fuse = train_eval(X_fused, "Fused (posture + CNN probs)")

rq3_tab1 = pd.DataFrame([row_post, row_spec, row_fuse])
rq3_tab1


✅ meta_df created: (450, 14)


,Model,Accuracy,F1,ROC-AUC
0,Posture-only,0.902655,0.867470,0.972260
1,Appearance-only (CNN probs),0.646018,0.047619,0.552055
2,Fused (posture + CNN probs),0.946903,0.926829,0.991781


In [10]:
# =========================
# RQ3: Outputs (2 figures + 2 tables)
# =========================

from sklearn.metrics import confusion_matrix

# ---- Figure 1: ROC-AUC comparison ----
fig = plt.figure(figsize=(7,4))
ax = fig.add_subplot(111)
ax.bar(rq3_tab1["Model"], rq3_tab1["ROC-AUC"])
ax.set_ylim(0, 1)
ax.set_ylabel("ROC-AUC")
ax.set_title("RQ3: Stress-risk proxy performance (unimodal vs fused)")
ax.tick_params(axis="x", rotation=20)
fig.savefig(FIG_ROOT/"RQ3"/"RQ3_Fig1.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Figure 2: Confusion matrix for fused model ----
cm3 = confusion_matrix(y[te_idx], pred_fuse)

fig = plt.figure(figsize=(4.5,4))
ax = fig.add_subplot(111)
ax.imshow(cm3)
ax.set_title("RQ3: Confusion matrix (Fused model)")
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(["Not-stress","Stress"])
ax.set_yticklabels(["Not-stress","Stress"])
for (i, j), v in np.ndenumerate(cm3):
    ax.text(j, i, str(v), ha="center", va="center")
fig.savefig(FIG_ROOT/"RQ3"/"RQ3_Fig2.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Table 1: model comparison ----
with pd.ExcelWriter(FIG_ROOT/"RQ3"/"RQ3_Tab1.xlsx") as w:
    rq3_tab1.to_excel(w, index=False, sheet_name="Model_Comparison")

# ---- Table 2: fused model coefficient importance ----
feature_names = ["crouch_score","norm_h","aspect"] + [c for c in meta_df.columns if c.startswith("p_")]
coefs = clf_fuse.named_steps["lr"].coef_.reshape(-1)

rq3_tab2 = (pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
    "abs_coef": np.abs(coefs)
}).sort_values("abs_coef", ascending=False).head(25))

with pd.ExcelWriter(FIG_ROOT/"RQ3"/"RQ3_Tab2.xlsx") as w:
    rq3_tab2.to_excel(w, index=False, sheet_name="Top_Coefficients")

print("✅ Saved RQ3_Fig1.pdf, RQ3_Fig2.pdf, RQ3_Tab1.xlsx, RQ3_Tab2.xlsx in:",
      (FIG_ROOT/"RQ3").resolve())


✅ Saved RQ3_Fig1.pdf, RQ3_Fig2.pdf, RQ3_Tab1.xlsx, RQ3_Tab2.xlsx in: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables\RQ3


## RQ4: Robustness under missing or noisy information

Real-world welfare monitoring (e.g., cameras in farms/shelters/homes) often suffers from partial views, occlusions, and uncertainty. This section tests robustness of the fused stress-risk proxy system under:

1) **Baseline fused** (all features available)  
2) **Missing posture** (posture features unavailable)  
3) **Low-confidence appearance** (CNN probabilities degraded when confidence is low)

We report performance changes using Accuracy, F1, and ROC-AUC.


In [11]:
# =========================
# RQ4: Robustness experiments
# =========================

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def eval_condition(X, label):
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(max_iter=400))
    ])
    clf.fit(X[tr_idx], y[tr_idx])

    pred = clf.predict(X[te_idx])
    proba = clf.predict_proba(X[te_idx])[:, 1]

    return {
        "Condition": label,
        "Accuracy": float(accuracy_score(y[te_idx], pred)),
        "F1": float(f1_score(y[te_idx], pred)),
        "ROC-AUC": float(roc_auc_score(y[te_idx], proba))
    }, pred, proba, clf

# 1) Baseline fused
row_base, pred_base, proba_base, clf_base = eval_condition(X_fused, "Baseline fused")

# 2) Missing posture: zero out crouch_score, norm_h, aspect (first 3 columns)
X_missing_posture = X_fused.copy()
X_missing_posture[:, :3] = 0
row_miss_post, pred_miss_post, proba_miss_post, clf_miss_post = eval_condition(X_missing_posture, "Missing posture")

# 3) Low-confidence appearance: if max class prob < threshold, zero out the prob vector
X_lowconf = X_fused.copy()
maxp = X_species.max(axis=1)
THRESH = 0.40
mask_low = maxp < THRESH
X_lowconf[mask_low, 3:] = 0
row_lowconf, pred_lowconf, proba_lowconf, clf_lowconf = eval_condition(X_lowconf, f"Low-confidence appearance (maxp<{THRESH})")

rq4_tab1 = pd.DataFrame([row_base, row_miss_post, row_lowconf])
rq4_tab1


,Condition,Accuracy,F1,ROC-AUC
0,Baseline fused,0.946903,0.926829,0.991781
1,Missing posture,0.646018,0.047619,0.552055
2,Low-confidence appearance (maxp<0.4),0.938053,0.915663,0.990411


In [12]:
# =========================
# RQ4: Outputs (2 figures + 2 tables)
# =========================

# ---- Figure 1: ROC-AUC by condition ----
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.bar(rq4_tab1["Condition"], rq4_tab1["ROC-AUC"])
ax.set_ylim(0,1)
ax.set_ylabel("ROC-AUC")
ax.set_title("RQ4: Robustness under missing/noisy inputs (ROC-AUC)")
ax.tick_params(axis="x", rotation=20)
fig.savefig(FIG_ROOT/"RQ4"/"RQ4_Fig1.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Figure 2: F1 by condition ----
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.bar(rq4_tab1["Condition"], rq4_tab1["F1"])
ax.set_ylim(0,1)
ax.set_ylabel("F1 Score")
ax.set_title("RQ4: Robustness under missing/noisy inputs (F1)")
ax.tick_params(axis="x", rotation=20)
fig.savefig(FIG_ROOT/"RQ4"/"RQ4_Fig2.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Table 1: Robustness performance ----
with pd.ExcelWriter(FIG_ROOT/"RQ4"/"RQ4_Tab1.xlsx") as w:
    rq4_tab1.to_excel(w, index=False, sheet_name="Robustness")

# ---- Table 2: Delta vs baseline ----
baseline = rq4_tab1.iloc[0].copy()
rq4_tab2 = rq4_tab1.copy()
rq4_tab2["Delta_Accuracy"] = rq4_tab2["Accuracy"] - float(baseline["Accuracy"])
rq4_tab2["Delta_F1"]       = rq4_tab2["F1"] - float(baseline["F1"])
rq4_tab2["Delta_ROC-AUC"]  = rq4_tab2["ROC-AUC"] - float(baseline["ROC-AUC"])

with pd.ExcelWriter(FIG_ROOT/"RQ4"/"RQ4_Tab2.xlsx") as w:
    rq4_tab2.to_excel(w, index=False, sheet_name="Delta_vs_Baseline")

print("✅ Saved RQ4_Fig1.pdf, RQ4_Fig2.pdf, RQ4_Tab1.xlsx, RQ4_Tab2.xlsx in:",
      (FIG_ROOT/"RQ4").resolve())


✅ Saved RQ4_Fig1.pdf, RQ4_Fig2.pdf, RQ4_Tab1.xlsx, RQ4_Tab2.xlsx in: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables\RQ4


## RQ5: Explainability and rule-based decision support for welfare screening

For animal welfare applications, predictions should be interpretable and actionable for end users (e.g., farmers, veterinarians, and pet owners). This section adds a simple **rule engine** that converts the stress-risk proxy into human-readable levels (Low/Moderate/High) and flags uncertain cases for **human review**.

The rule engine uses:
- a posture-based threshold (within-species crouch-score percentile), and
- a confidence heuristic based on CNN probability strength.


In [13]:
# =========================
# RQ5: Rule engine + explainability tables
# =========================

# We assume meta_df exists from RQ3 and includes:
# crouch_score, species, and p_* columns, plus stress_proxy

prob_cols = [c for c in meta_df.columns if c.startswith("p_")]
if len(prob_cols) == 0:
    raise RuntimeError("No probability columns found in meta_df. Rerun RQ3.")

# Threshold map: within each species, compute the 70th percentile of crouch_score
q70_map = meta_df.groupby("species")["crouch_score"].quantile(0.70).to_dict()

def topk_classes(prob_vec, k=3):
    idx = np.argsort(prob_vec)[::-1][:k]
    return [(class_names[i], float(prob_vec[i])) for i in idx]

def rule_engine(species, crouch_score, max_prob):
    """
    Simple interpretable rule layer:
    - 'stressed_proxy' if crouch_score >= species-specific threshold
    - 'human_review' if max_prob is low (uncertain classification)
    - rule_level: Low / Moderate / High
    """
    stressed_proxy = crouch_score >= q70_map.get(species, 1.0)
    human_review = max_prob < 0.40

    if stressed_proxy and human_review:
        level = "High"
    elif stressed_proxy:
        level = "Moderate"
    else:
        level = "Low"

    return level, int(human_review), int(stressed_proxy)

# Apply rule engine to all rows
levels, reviews, stressed_flags = [], [], []
top3_list = []

for i in range(len(meta_df)):
    sp = meta_df.loc[i, "species"]
    cs = float(meta_df.loc[i, "crouch_score"])
    probs = meta_df.loc[i, prob_cols].to_numpy()
    max_prob = float(probs.max())

    lvl, rev, st = rule_engine(sp, cs, max_prob)
    levels.append(lvl)
    reviews.append(rev)
    stressed_flags.append(st)
    top3_list.append(str(topk_classes(probs, 3)))

meta_df["rule_level"] = levels
meta_df["human_review"] = reviews
meta_df["stressed_proxy_flag"] = stressed_flags
meta_df["top3_probs"] = top3_list

# Summary tables
rq5_tab1 = meta_df.groupby("rule_level").size().reset_index(name="count")
rq5_tab2 = (pd.crosstab(meta_df["species"], meta_df["rule_level"], normalize="index")
            .reset_index())

# Examples table (small)
rq5_examples = meta_df[["species","crouch_score","stressed_proxy_flag","rule_level","human_review","top3_probs"]].head(20)

print("✅ Rule level counts:")
display(rq5_tab1)

print("✅ Example rows:")
display(rq5_examples.head())


✅ Rule level counts:


,rule_level,count
0,High,21
1,Low,292
2,Moderate,137


✅ Example rows:


,species,crouch_score,stressed_proxy_flag,rule_level,human_review,top3_probs
0,pecora,0.107143,0,Low,0,"[('cane', 0.5379108190536499), ('scoiattolo', ..."
1,mucca,0.182927,1,High,1,"[('gallina', 0.2859024107456207), ('farfalla',..."
2,pecora,0.133333,0,Low,0,"[('cane', 0.6948132514953613), ('pecora', 0.10..."
3,ragno,0.116732,0,Low,0,"[('ragno', 0.6813032627105713), ('farfalla', 0..."
4,cavallo,0.080333,0,Low,0,"[('cane', 0.6530382633209229), ('cavallo', 0.1..."


In [14]:
# =========================
# RQ5: Outputs (2 figures + 2 tables)
# =========================

# ---- Figure 1: Rule-level distribution ----
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.bar(rq5_tab1["rule_level"], rq5_tab1["count"])
ax.set_title("RQ5: Rule-based stress-risk level distribution (proxy)")
ax.set_xlabel("Rule level")
ax.set_ylabel("Count")
fig.savefig(FIG_ROOT/"RQ5"/"RQ5_Fig1.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Figure 2: Human review flags ----
review_counts = meta_df["human_review"].value_counts().sort_index()
no_review = int(review_counts.get(0, 0))
needs_review = int(review_counts.get(1, 0))

fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.bar(["No review", "Needs review"], [no_review, needs_review])
ax.set_title("RQ5: Human-review flags (uncertainty handling)")
ax.set_ylabel("Count")
fig.savefig(FIG_ROOT/"RQ5"/"RQ5_Fig2.pdf", bbox_inches="tight")
plt.close(fig)

# ---- Table 1: Rule summary + examples + review rate ----
review_rate = float(meta_df["human_review"].mean())
review_rate_df = pd.DataFrame([{"human_review_rate": review_rate}])

with pd.ExcelWriter(FIG_ROOT/"RQ5"/"RQ5_Tab1.xlsx") as w:
    rq5_tab1.to_excel(w, index=False, sheet_name="RuleLevel_Counts")
    review_rate_df.to_excel(w, index=False, sheet_name="ReviewRate")
    rq5_examples.to_excel(w, index=False, sheet_name="Examples")

# ---- Table 2: Rule levels by species (row-normalized) ----
with pd.ExcelWriter(FIG_ROOT/"RQ5"/"RQ5_Tab2.xlsx") as w:
    rq5_tab2.to_excel(w, index=False, sheet_name="RuleLevel_BySpecies")

print("✅ Saved RQ5_Fig1.pdf, RQ5_Fig2.pdf, RQ5_Tab1.xlsx, RQ5_Tab2.xlsx in:",
      (FIG_ROOT/"RQ5").resolve())


✅ Saved RQ5_Fig1.pdf, RQ5_Fig2.pdf, RQ5_Tab1.xlsx, RQ5_Tab2.xlsx in: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables\RQ5


In [15]:
# =========================
# Final Compliance Check
# =========================

from pathlib import Path

required = []
for rq in range(1, 6):
    folder = FIG_ROOT / f"RQ{rq}"
    required.extend([
        folder / f"RQ{rq}_Fig1.pdf",
        folder / f"RQ{rq}_Fig2.pdf",
        folder / f"RQ{rq}_Tab1.xlsx",
        folder / f"RQ{rq}_Tab2.xlsx",
    ])

missing = [str(p) for p in required if not p.exists()]

if missing:
    print("❌ Missing required files:")
    for m in missing:
        print(" -", m)
else:
    print("✅ ALL REQUIRED FILES EXIST")
    print("Total files checked:", len(required))


✅ ALL REQUIRED FILES EXIST
Total files checked: 20


In [17]:
# =========================
# Create Figures_Tables ZIP
# =========================

import zipfile

zip_path = BASE_DIR / "Figures_Tables.zip"

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for rq_dir in FIG_ROOT.iterdir():
        if rq_dir.is_dir():
            for f in rq_dir.iterdir():
                z.write(f, arcname=f"Figures_Tables/{rq_dir.name}/{f.name}")

print("✅ ZIP created at:", zip_path.resolve())


✅ ZIP created at: C:\Users\nitro\Documents\animals10_project\outputs_part2_animals10\Figures_Tables.zip
